!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!This script contains maskign code before standardisation. however standardisation of inputs and targets was performed before the interpoléated baselines were generated via CDO for bilinear and bicubic and then masked so that they only retain the Swiss Domain

Workflow : Bilinear interpolation, bicubic interpolation, Masking to retain only the swiss domain, splitting into train, test val, gridded RMSE R2, pooled RMSE, R2

In [1]:
import sys
sys.path.append("/work/FAC/FGSE/IDYST/tbeucler/downscaling/sasthana/Downscaling/Downscaling/Scripts/Functions")
import xarray as xr
import numpy as np

In [2]:
from dask.distributed import Client
client= Client()
print(client)

<Client: 'tcp://127.0.0.1:38717' processes=1 threads=1, memory=503.49 GiB>


installed dask distributed, and used all 4 CPU cores, computing time reduced to 30 seconds per line!!!

xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxCHECKING SHAPExxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx

FEATURES : CHECKIGN SHAPES

In [6]:

features_ds_precip = xr.open_dataset("/work/FAC/FGSE/IDYST/tbeucler/downscaling/sasthana/Downscaling/Downscaling/data/Features/bicubic_rhiresd.nc")["RhiresD"]
features_ds_precip.shape

FileNotFoundError: [Errno 2] No such file or directory: '/work/FAC/FGSE/IDYST/tbeucler/downscaling/sasthana/Downscaling/Downscaling/data/Features/bicubic_rhiresd.nc'

In [5]:
features_ds_tabsd= xr.open_dataset("/work/FAC/FGSE/IDYST/tbeucler/downscaling/sasthana/Downscaling/Downscaling/data/Features/bicubic_tabsd.nc")["TabsD"]
features_ds_tabsd.shape

(19358, 240, 370)

In [3]:
features_ds_tmax= xr.open_dataset("/work/FAC/FGSE/IDYST/tbeucler/downscaling/sasthana/Downscaling/Downscaling/data/Features/bicubic_tmax.nc")["TmaxD"]
features_ds_tmax.shape

FileNotFoundError: [Errno 2] No such file or directory: '/work/FAC/FGSE/IDYST/tbeucler/downscaling/sasthana/Downscaling/Downscaling/data/Features/bicubic_tmax.nc'

In [7]:
features_ds_tmin= xr.open_dataset("/work/FAC/FGSE/IDYST/tbeucler/downscaling/sasthana/Downscaling/Downscaling/data/Features/bicubic_tmin.nc")["TminD"]
features_ds_tmin.shape

(19358, 240, 370)

CHECKING SHAPES OF TARGETS

In [6]:
targets_ds_precip = xr.open_dataset("/work/FAC/FGSE/IDYST/tbeucler/downscaling/sasthana/Downscaling/Downscaling/data/Targets/files/RhiresD_wet_1971_2023.nc")["RhiresD"]
targets_ds_precip.shape

(19358, 240, 370)

In [7]:
targets_ds_tabsd= xr.open_dataset("/work/FAC/FGSE/IDYST/tbeucler/downscaling/sasthana/Downscaling/Downscaling/data/Targets/files/TabsD_wet_1971_2023.nc")["TabsD"]
targets_ds_tabsd.shape

(19358, 240, 370)

In [3]:
targets_ds_tmax= xr.open_dataset("/work/FAC/FGSE/IDYST/tbeucler/downscaling/sasthana/Downscaling/Downscaling/data/Targets/files/TmaxD_wet_1971_2023.nc")["TmaxD"]

In [9]:
targets_ds_tmin= xr.open_dataset("/work/FAC/FGSE/IDYST/tbeucler/downscaling/sasthana/Downscaling/Downscaling/data/Targets/files/TminD_wet_1971_2023.nc")["TminD"]
targets_ds_tmin.shape

(19358, 240, 370)

SHAPES MATCH!!!!!!

The entire dataset will be divided into tain, test val for both bilinear and bicubic interpolation for calculating standardisation metric etc


All splits performed according to the function written in Functions/Train_Test_Val.py

In [3]:
from Train_Test_Val import split_by_decade

Splitting features

RhiresD, TabsD, Tmin, Tmax

Example Usage

In [ ]:

train, val, test = split_by_decade(
    features_ds_precip,
    "/work/FAC/FGSE/IDYST/tbeucler/downscaling/sasthana/Downscaling/Downscaling/data/Split_Data",
    "rhiresd_features"
)


Splitting targets along the same lines

Example Usage

In [4]:
targets_ds_tmax = xr.open_dataset(
    "/work/FAC/FGSE/IDYST/tbeucler/downscaling/sasthana/Downscaling/Downscaling/data/Targets/files/TmaxD_wet_1971_2023.nc",
    chunks={"time": 365})["TmaxD"]

train, val, test = split_by_decade(
    targets_ds_tmax,
    "/work/FAC/FGSE/IDYST/tbeucler/downscaling/sasthana/Downscaling/Downscaling/data/Targets/Split_Data",
    "tmax_targets"
)

Also splittign the targets along the very same lines usign the reproducible split

xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxDoing the same train test val split for bilinear nterpolation xxxxxxxxx

Standardisation code

Standardisation has to be performed avoiding data leakage. Hence, the train -test contamination has to be avoided. 

Can be done using statistics only for the training set

The mean, std, min, max and range of the training set will then be used to perform scaling for the test set as well. 

xxxxxxxxxxxxPerforming standardisation for the training set for both precip and temperaturexxxxx. Training will be performed on the bicubically interpolated dataset for now

In [22]:
from Standardise import standardise

Temperature 

Loading inputs and targets once again separately

#Scaling Precipitation training features and targets (from bicubic interpolation features): targets and features get scaled by their own mean and std. The same parametersa then used to standardise testing and validation

In [23]:
input_path = "/work/FAC/FGSE/IDYST/tbeucler/downscaling/sasthana/Downscaling/Downscaling/data/processed/Bicubic/Train/features_precip_masked_bicubic_train.nc"
output_path = "/work/FAC/FGSE/IDYST/tbeucler/downscaling/sasthana/Downscaling/Downscaling/data/processed/Bicubic/Train/SCALED_features_precip_masked_bicubic_train.nc"
var = "pr"
precip_min, precip_max = standardise(input_path, output_path, var)

/work/FAC/FGSE/IDYST/tbeucler/downscaling/sasthana/MyPythonEnvironment/lib/python3.12/site-packages/dask/_task_spec.py:758: RuntimeWarning: invalid value encountered in divide
  return self.func(*new_argspec)
/work/FAC/FGSE/IDYST/tbeucler/downscaling/sasthana/MyPythonEnvironment/lib/python3.12/site-packages/dask/_task_spec.py:758: RuntimeWarning: invalid value encountered in divide
  return self.func(*new_argspec)
/work/FAC/FGSE/IDYST/tbeucler/downscaling/sasthana/MyPythonEnvironment/lib/python3.12/site-packages/dask/_task_spec.py:758: RuntimeWarning: invalid value encountered in divide
  return self.func(*new_argspec)
/work/FAC/FGSE/IDYST/tbeucler/downscaling/sasthana/MyPythonEnvironment/lib/python3.12/site-packages/dask/_task_spec.py:758: RuntimeWarning: invalid value encountered in divide
  return self.func(*new_argspec)


Normalised/scaled variable saved to /work/FAC/FGSE/IDYST/tbeucler/downscaling/sasthana/Downscaling/Downscaling/data/processed/Bicubic/Train/SCALED_features_precip_masked_bicubic_train.nc


In [12]:
input_path = "/work/FAC/FGSE/IDYST/tbeucler/downscaling/sasthana/Downscaling/Downscaling/data/processed/Bicubic/Train/targets_precip_masked_train.nc"
output_path = "/work/FAC/FGSE/IDYST/tbeucler/downscaling/sasthana/Downscaling/Downscaling/data/processed/Bicubic/Train/SCALED_targets_precip_masked_train.nc"
var = "RhiresD"
target_min, target_max = standardise(input_path, output_path, var)

Normalised/scaled variable saved to /work/FAC/FGSE/IDYST/tbeucler/downscaling/sasthana/Downscaling/Downscaling/data/processed/Bicubic/Train/SCALED_targets_precip_masked_train.nc


Using above mean, max, std and min from temp and precip to standardise the vcalidation and testing data to avoid data leakage

xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxPRECIPITATIONxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx

In [13]:
# Validation features :Precip
input_path = "/work/FAC/FGSE/IDYST/tbeucler/downscaling/sasthana/Downscaling/Downscaling/data/processed/Bicubic/Val/features_precip_masked_bicubic_val.nc"
output_path = "/work/FAC/FGSE/IDYST/tbeucler/downscaling/sasthana/Downscaling/Downscaling/data/processed/Bicubic/Val/SCALED_features_precip_masked_bicubic_val.nc"
var = "pr"
standardise(input_path, output_path, var, min=precip_min, max=precip_max)

/work/FAC/FGSE/IDYST/tbeucler/downscaling/sasthana/MyPythonEnvironment/lib/python3.12/site-packages/dask/_task_spec.py:758: RuntimeWarning: invalid value encountered in divide
  return self.func(*new_argspec)
/work/FAC/FGSE/IDYST/tbeucler/downscaling/sasthana/MyPythonEnvironment/lib/python3.12/site-packages/dask/_task_spec.py:758: RuntimeWarning: invalid value encountered in divide
  return self.func(*new_argspec)
/work/FAC/FGSE/IDYST/tbeucler/downscaling/sasthana/MyPythonEnvironment/lib/python3.12/site-packages/dask/_task_spec.py:758: RuntimeWarning: invalid value encountered in divide
  return self.func(*new_argspec)
/work/FAC/FGSE/IDYST/tbeucler/downscaling/sasthana/MyPythonEnvironment/lib/python3.12/site-packages/dask/_task_spec.py:758: RuntimeWarning: invalid value encountered in divide
  return self.func(*new_argspec)


Normalised/scaled variable saved to /work/FAC/FGSE/IDYST/tbeucler/downscaling/sasthana/Downscaling/Downscaling/data/processed/Bicubic/Val/SCALED_features_precip_masked_bicubic_val.nc


(<xarray.DataArray 'pr' (N: 265, E: 370)> Size: 784kB
 dask.array<_nanmin_skip-aggregate, shape=(265, 370), dtype=float64, chunksize=(265, 370), chunktype=numpy.ndarray>
 Coordinates:
     lon      (N, E) float32 392kB dask.array<chunksize=(265, 370), meta=np.ndarray>
     lat      (N, E) float32 392kB dask.array<chunksize=(265, 370), meta=np.ndarray>
   * E        (E) float64 3kB 2.474e+06 2.476e+06 ... 2.842e+06 2.844e+06
   * N        (N) float64 2kB 1.06e+06 1.06e+06 1.062e+06 ... 1.322e+06 1.324e+06,
 <xarray.DataArray 'pr' (N: 265, E: 370)> Size: 784kB
 dask.array<_nanmax_skip-aggregate, shape=(265, 370), dtype=float64, chunksize=(265, 370), chunktype=numpy.ndarray>
 Coordinates:
     lon      (N, E) float32 392kB dask.array<chunksize=(265, 370), meta=np.ndarray>
     lat      (N, E) float32 392kB dask.array<chunksize=(265, 370), meta=np.ndarray>
   * E        (E) float64 3kB 2.474e+06 2.476e+06 ... 2.842e+06 2.844e+06
   * N        (N) float64 2kB 1.06e+06 1.06e+06 1.062e+06 ...

In [14]:

# Validation targets:  Precip
input_path = "/work/FAC/FGSE/IDYST/tbeucler/downscaling/sasthana/Downscaling/Downscaling/data/processed/Bicubic/Val/targets_precip_masked_val.nc"
output_path = "/work/FAC/FGSE/IDYST/tbeucler/downscaling/sasthana/Downscaling/Downscaling/data/processed/Bicubic/Val/SCALED_targets_precip_masked_val.nc"
var = "RhiresD"
standardise(input_path, output_path, var, min=target_min, max=target_max)

Normalised/scaled variable saved to /work/FAC/FGSE/IDYST/tbeucler/downscaling/sasthana/Downscaling/Downscaling/data/processed/Bicubic/Val/SCALED_targets_precip_masked_val.nc


(<xarray.DataArray 'RhiresD' (N: 265, E: 370)> Size: 392kB
 dask.array<_nanmin_skip-aggregate, shape=(265, 370), dtype=float32, chunksize=(265, 370), chunktype=numpy.ndarray>
 Coordinates:
     lon      (N, E) float32 392kB dask.array<chunksize=(265, 370), meta=np.ndarray>
     lat      (N, E) float32 392kB dask.array<chunksize=(265, 370), meta=np.ndarray>
   * E        (E) float64 3kB 2.474e+06 2.476e+06 ... 2.842e+06 2.844e+06
   * N        (N) float64 2kB 1.06e+06 1.06e+06 1.062e+06 ... 1.322e+06 1.324e+06,
 <xarray.DataArray 'RhiresD' (N: 265, E: 370)> Size: 392kB
 dask.array<_nanmax_skip-aggregate, shape=(265, 370), dtype=float32, chunksize=(265, 370), chunktype=numpy.ndarray>
 Coordinates:
     lon      (N, E) float32 392kB dask.array<chunksize=(265, 370), meta=np.ndarray>
     lat      (N, E) float32 392kB dask.array<chunksize=(265, 370), meta=np.ndarray>
   * E        (E) float64 3kB 2.474e+06 2.476e+06 ... 2.842e+06 2.844e+06
   * N        (N) float64 2kB 1.06e+06 1.06e+06 1.0

In [15]:
# Precipitation test features
input_path = "/work/FAC/FGSE/IDYST/tbeucler/downscaling/sasthana/Downscaling/Downscaling/data/processed/Bicubic/Test/features_precip_masked_bicubic_test.nc"
output_path = "/work/FAC/FGSE/IDYST/tbeucler/downscaling/sasthana/Downscaling/Downscaling/data/processed/Bicubic/Test/SCALED_features_precip_masked_bicubic_test.nc"
var = "pr"
standardise(input_path, output_path, var, min=precip_min, max=precip_max)

/work/FAC/FGSE/IDYST/tbeucler/downscaling/sasthana/MyPythonEnvironment/lib/python3.12/site-packages/dask/_task_spec.py:758: RuntimeWarning: invalid value encountered in divide
  return self.func(*new_argspec)
/work/FAC/FGSE/IDYST/tbeucler/downscaling/sasthana/MyPythonEnvironment/lib/python3.12/site-packages/dask/_task_spec.py:758: RuntimeWarning: invalid value encountered in divide
  return self.func(*new_argspec)
/work/FAC/FGSE/IDYST/tbeucler/downscaling/sasthana/MyPythonEnvironment/lib/python3.12/site-packages/dask/_task_spec.py:758: RuntimeWarning: invalid value encountered in divide
  return self.func(*new_argspec)
/work/FAC/FGSE/IDYST/tbeucler/downscaling/sasthana/MyPythonEnvironment/lib/python3.12/site-packages/dask/_task_spec.py:758: RuntimeWarning: invalid value encountered in divide
  return self.func(*new_argspec)


Normalised/scaled variable saved to /work/FAC/FGSE/IDYST/tbeucler/downscaling/sasthana/Downscaling/Downscaling/data/processed/Bicubic/Test/SCALED_features_precip_masked_bicubic_test.nc


(<xarray.DataArray 'pr' (N: 265, E: 370)> Size: 784kB
 dask.array<_nanmin_skip-aggregate, shape=(265, 370), dtype=float64, chunksize=(265, 370), chunktype=numpy.ndarray>
 Coordinates:
     lon      (N, E) float32 392kB dask.array<chunksize=(265, 370), meta=np.ndarray>
     lat      (N, E) float32 392kB dask.array<chunksize=(265, 370), meta=np.ndarray>
   * E        (E) float64 3kB 2.474e+06 2.476e+06 ... 2.842e+06 2.844e+06
   * N        (N) float64 2kB 1.06e+06 1.06e+06 1.062e+06 ... 1.322e+06 1.324e+06,
 <xarray.DataArray 'pr' (N: 265, E: 370)> Size: 784kB
 dask.array<_nanmax_skip-aggregate, shape=(265, 370), dtype=float64, chunksize=(265, 370), chunktype=numpy.ndarray>
 Coordinates:
     lon      (N, E) float32 392kB dask.array<chunksize=(265, 370), meta=np.ndarray>
     lat      (N, E) float32 392kB dask.array<chunksize=(265, 370), meta=np.ndarray>
   * E        (E) float64 3kB 2.474e+06 2.476e+06 ... 2.842e+06 2.844e+06
   * N        (N) float64 2kB 1.06e+06 1.06e+06 1.062e+06 ...

In [16]:
# Precipitation test targets
input_path = "/work/FAC/FGSE/IDYST/tbeucler/downscaling/sasthana/Downscaling/Downscaling/data/processed/Bicubic/Test/targets_precip_masked_test.nc"
output_path = "/work/FAC/FGSE/IDYST/tbeucler/downscaling/sasthana/Downscaling/Downscaling/data/processed/Bicubic/Test/SCALED_targets_precip_masked_test.nc"
var = "RhiresD"
standardise(input_path, output_path, var, min=target_min, max=target_max)

Normalised/scaled variable saved to /work/FAC/FGSE/IDYST/tbeucler/downscaling/sasthana/Downscaling/Downscaling/data/processed/Bicubic/Test/SCALED_targets_precip_masked_test.nc


(<xarray.DataArray 'RhiresD' (N: 265, E: 370)> Size: 392kB
 dask.array<_nanmin_skip-aggregate, shape=(265, 370), dtype=float32, chunksize=(265, 370), chunktype=numpy.ndarray>
 Coordinates:
     lon      (N, E) float32 392kB dask.array<chunksize=(265, 370), meta=np.ndarray>
     lat      (N, E) float32 392kB dask.array<chunksize=(265, 370), meta=np.ndarray>
   * E        (E) float64 3kB 2.474e+06 2.476e+06 ... 2.842e+06 2.844e+06
   * N        (N) float64 2kB 1.06e+06 1.06e+06 1.062e+06 ... 1.322e+06 1.324e+06,
 <xarray.DataArray 'RhiresD' (N: 265, E: 370)> Size: 392kB
 dask.array<_nanmax_skip-aggregate, shape=(265, 370), dtype=float32, chunksize=(265, 370), chunktype=numpy.ndarray>
 Coordinates:
     lon      (N, E) float32 392kB dask.array<chunksize=(265, 370), meta=np.ndarray>
     lat      (N, E) float32 392kB dask.array<chunksize=(265, 370), meta=np.ndarray>
   * E        (E) float64 3kB 2.474e+06 2.476e+06 ... 2.842e+06 2.844e+06
   * N        (N) float64 2kB 1.06e+06 1.06e+06 1.0

xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxTEMPERATURE : Val and test targets and features standardisationxxxxxxxxxxxxxxxx


Training features 

In [17]:
tas_mean, tas_std = standardise(
    input_path="/work/FAC/FGSE/IDYST/tbeucler/downscaling/sasthana/Downscaling/Downscaling/data/processed/Bicubic/Train/features_tas_masked_bicubic_train.nc",
    output_path="/work/FAC/FGSE/IDYST/tbeucler/downscaling/sasthana/Downscaling/Downscaling/data/processed/Bicubic/Train/SCALED_features_tas_masked_bicubic_train.nc",
    var="tas"
)

/work/FAC/FGSE/IDYST/tbeucler/downscaling/sasthana/MyPythonEnvironment/lib/python3.12/site-packages/dask/_task_spec.py:758: RuntimeWarning: invalid value encountered in divide
  return self.func(*new_argspec)
/work/FAC/FGSE/IDYST/tbeucler/downscaling/sasthana/MyPythonEnvironment/lib/python3.12/site-packages/dask/_task_spec.py:758: RuntimeWarning: invalid value encountered in divide
  return self.func(*new_argspec)
/work/FAC/FGSE/IDYST/tbeucler/downscaling/sasthana/MyPythonEnvironment/lib/python3.12/site-packages/dask/_task_spec.py:758: RuntimeWarning: invalid value encountered in divide
  return self.func(*new_argspec)
/work/FAC/FGSE/IDYST/tbeucler/downscaling/sasthana/MyPythonEnvironment/lib/python3.12/site-packages/dask/_task_spec.py:758: RuntimeWarning: invalid value encountered in divide
  return self.func(*new_argspec)


Normalised/scaled variable saved to /work/FAC/FGSE/IDYST/tbeucler/downscaling/sasthana/Downscaling/Downscaling/data/processed/Bicubic/Train/SCALED_features_tas_masked_bicubic_train.nc


Training targets 

In [18]:
tabsd_mean, tabsd_std = standardise(
    input_path="/work/FAC/FGSE/IDYST/tbeucler/downscaling/sasthana/Downscaling/Downscaling/data/processed/Bicubic/Train/targets_tas_masked_train.nc",
    output_path="/work/FAC/FGSE/IDYST/tbeucler/downscaling/sasthana/Downscaling/Downscaling/data/processed/Bicubic/Train/SCALED_targets_tas_masked_train.nc",
    var="TabsD"
)

/work/FAC/FGSE/IDYST/tbeucler/downscaling/sasthana/MyPythonEnvironment/lib/python3.12/site-packages/dask/array/numpy_compat.py:57: RuntimeWarning: invalid value encountered in divide
  x = np.divide(x1, x2, out)


Normalised/scaled variable saved to /work/FAC/FGSE/IDYST/tbeucler/downscaling/sasthana/Downscaling/Downscaling/data/processed/Bicubic/Train/SCALED_targets_tas_masked_train.nc


Now we have the parameters from the training set for temperature, the same will be used to scale the featurea and targets of the Val and Test set

Validation standardisation

In [20]:
standardise(
    input_path="/work/FAC/FGSE/IDYST/tbeucler/downscaling/sasthana/Downscaling/Downscaling/data/processed/Bicubic/Val/features_tas_masked_bicubic_val.nc",
    output_path="/work/FAC/FGSE/IDYST/tbeucler/downscaling/sasthana/Downscaling/Downscaling/data/processed/Bicubic/Val/SCALED_features_tas_masked_bicubic_val.nc",
    var="tas",
    mean=tas_mean,
    std=tas_std
)

Normalised/scaled variable saved to /work/FAC/FGSE/IDYST/tbeucler/downscaling/sasthana/Downscaling/Downscaling/data/processed/Bicubic/Val/SCALED_features_tas_masked_bicubic_val.nc


(<xarray.DataArray 'tas' (N: 265, E: 370)> Size: 784kB
 dask.array<mean_agg-aggregate, shape=(265, 370), dtype=float64, chunksize=(265, 370), chunktype=numpy.ndarray>
 Coordinates:
     lon      (N, E) float32 392kB dask.array<chunksize=(265, 370), meta=np.ndarray>
     lat      (N, E) float32 392kB dask.array<chunksize=(265, 370), meta=np.ndarray>
   * E        (E) float64 3kB 2.474e+06 2.476e+06 ... 2.842e+06 2.844e+06
   * N        (N) float64 2kB 1.06e+06 1.06e+06 1.062e+06 ... 1.322e+06 1.324e+06,
 <xarray.DataArray 'tas' (N: 265, E: 370)> Size: 784kB
 dask.array<_sqrt, shape=(265, 370), dtype=float64, chunksize=(265, 370), chunktype=numpy.ndarray>
 Coordinates:
     lon      (N, E) float32 392kB dask.array<chunksize=(265, 370), meta=np.ndarray>
     lat      (N, E) float32 392kB dask.array<chunksize=(265, 370), meta=np.ndarray>
   * E        (E) float64 3kB 2.474e+06 2.476e+06 ... 2.842e+06 2.844e+06
   * N        (N) float64 2kB 1.06e+06 1.06e+06 1.062e+06 ... 1.322e+06 1.324e+0

In [21]:
standardise(
    input_path="/work/FAC/FGSE/IDYST/tbeucler/downscaling/sasthana/Downscaling/Downscaling/data/processed/Bicubic/Val/targets_tas_masked_val.nc",
    output_path="/work/FAC/FGSE/IDYST/tbeucler/downscaling/sasthana/Downscaling/Downscaling/data/processed/Bicubic/Val/SCALED_targets_tas_masked_val.nc",
    var="TabsD",
    mean=tabsd_mean,
    std=tabsd_std
)

/work/FAC/FGSE/IDYST/tbeucler/downscaling/sasthana/MyPythonEnvironment/lib/python3.12/site-packages/dask/array/numpy_compat.py:57: RuntimeWarning: invalid value encountered in divide
  x = np.divide(x1, x2, out)


Normalised/scaled variable saved to /work/FAC/FGSE/IDYST/tbeucler/downscaling/sasthana/Downscaling/Downscaling/data/processed/Bicubic/Val/SCALED_targets_tas_masked_val.nc


(<xarray.DataArray 'TabsD' (N: 265, E: 370)> Size: 392kB
 dask.array<mean_agg-aggregate, shape=(265, 370), dtype=float32, chunksize=(265, 370), chunktype=numpy.ndarray>
 Coordinates:
     lon      (N, E) float32 392kB dask.array<chunksize=(265, 370), meta=np.ndarray>
     lat      (N, E) float32 392kB dask.array<chunksize=(265, 370), meta=np.ndarray>
   * E        (E) float64 3kB 2.474e+06 2.476e+06 ... 2.842e+06 2.844e+06
   * N        (N) float64 2kB 1.06e+06 1.06e+06 1.062e+06 ... 1.322e+06 1.324e+06,
 <xarray.DataArray 'TabsD' (N: 265, E: 370)> Size: 392kB
 dask.array<_sqrt, shape=(265, 370), dtype=float32, chunksize=(265, 370), chunktype=numpy.ndarray>
 Coordinates:
     lon      (N, E) float32 392kB dask.array<chunksize=(265, 370), meta=np.ndarray>
     lat      (N, E) float32 392kB dask.array<chunksize=(265, 370), meta=np.ndarray>
   * E        (E) float64 3kB 2.474e+06 2.476e+06 ... 2.842e+06 2.844e+06
   * N        (N) float64 2kB 1.06e+06 1.06e+06 1.062e+06 ... 1.322e+06 1.32

Testing datasets for temepratures : standardisation of features and targets 

In [22]:
standardise(
    input_path="/work/FAC/FGSE/IDYST/tbeucler/downscaling/sasthana/Downscaling/Downscaling/data/processed/Bicubic/Test/features_tas_masked_bicubic_test.nc",
    output_path="/work/FAC/FGSE/IDYST/tbeucler/downscaling/sasthana/Downscaling/Downscaling/data/processed/Bicubic/Test/SCALED_features_tas_masked_bicubic_test.nc",
    var="tas",
    mean=tas_mean,
    std=tas_std
)

Normalised/scaled variable saved to /work/FAC/FGSE/IDYST/tbeucler/downscaling/sasthana/Downscaling/Downscaling/data/processed/Bicubic/Test/SCALED_features_tas_masked_bicubic_test.nc


(<xarray.DataArray 'tas' (N: 265, E: 370)> Size: 784kB
 dask.array<mean_agg-aggregate, shape=(265, 370), dtype=float64, chunksize=(265, 370), chunktype=numpy.ndarray>
 Coordinates:
     lon      (N, E) float32 392kB dask.array<chunksize=(265, 370), meta=np.ndarray>
     lat      (N, E) float32 392kB dask.array<chunksize=(265, 370), meta=np.ndarray>
   * E        (E) float64 3kB 2.474e+06 2.476e+06 ... 2.842e+06 2.844e+06
   * N        (N) float64 2kB 1.06e+06 1.06e+06 1.062e+06 ... 1.322e+06 1.324e+06,
 <xarray.DataArray 'tas' (N: 265, E: 370)> Size: 784kB
 dask.array<_sqrt, shape=(265, 370), dtype=float64, chunksize=(265, 370), chunktype=numpy.ndarray>
 Coordinates:
     lon      (N, E) float32 392kB dask.array<chunksize=(265, 370), meta=np.ndarray>
     lat      (N, E) float32 392kB dask.array<chunksize=(265, 370), meta=np.ndarray>
   * E        (E) float64 3kB 2.474e+06 2.476e+06 ... 2.842e+06 2.844e+06
   * N        (N) float64 2kB 1.06e+06 1.06e+06 1.062e+06 ... 1.322e+06 1.324e+0

In [23]:
standardise(
    input_path="/work/FAC/FGSE/IDYST/tbeucler/downscaling/sasthana/Downscaling/Downscaling/data/processed/Bicubic/Test/targets_tas_masked_test.nc",
    output_path="/work/FAC/FGSE/IDYST/tbeucler/downscaling/sasthana/Downscaling/Downscaling/data/processed/Bicubic/Test/SCALED_targets_tas_masked_test.nc",
    var="TabsD",
    mean=tabsd_mean,
    std=tabsd_std
)

Normalised/scaled variable saved to /work/FAC/FGSE/IDYST/tbeucler/downscaling/sasthana/Downscaling/Downscaling/data/processed/Bicubic/Test/SCALED_targets_tas_masked_test.nc


(<xarray.DataArray 'TabsD' (N: 265, E: 370)> Size: 392kB
 dask.array<mean_agg-aggregate, shape=(265, 370), dtype=float32, chunksize=(265, 370), chunktype=numpy.ndarray>
 Coordinates:
     lon      (N, E) float32 392kB dask.array<chunksize=(265, 370), meta=np.ndarray>
     lat      (N, E) float32 392kB dask.array<chunksize=(265, 370), meta=np.ndarray>
   * E        (E) float64 3kB 2.474e+06 2.476e+06 ... 2.842e+06 2.844e+06
   * N        (N) float64 2kB 1.06e+06 1.06e+06 1.062e+06 ... 1.322e+06 1.324e+06,
 <xarray.DataArray 'TabsD' (N: 265, E: 370)> Size: 392kB
 dask.array<_sqrt, shape=(265, 370), dtype=float32, chunksize=(265, 370), chunktype=numpy.ndarray>
 Coordinates:
     lon      (N, E) float32 392kB dask.array<chunksize=(265, 370), meta=np.ndarray>
     lat      (N, E) float32 392kB dask.array<chunksize=(265, 370), meta=np.ndarray>
   * E        (E) float64 3kB 2.474e+06 2.476e+06 ... 2.842e+06 2.844e+06
   * N        (N) float64 2kB 1.06e+06 1.06e+06 1.062e+06 ... 1.322e+06 1.32